# Novelty detection for credit card customers using Isolation Forest

In this project I wrote a code for **anomaly detection** in a credit card dataset from Kaggle.com. The dataset consisted of anonymized credit card transactions labeled as fraudulent or genuine.  
I used the **novelty detection method**, where the training set only had examples with class=0, to identify unusual patterns that do not conform to expected behavior. To implement this, I used the **Isolation Forest model**, which is a type of unsupervised machine learning algorithm that is used for anomaly detection.  
Finally, I used **Grid search** to tune the parameters, which allowed me to find the best combination of hyperparameters for the model.  
Anomaly detection is important in many applications, including detecting fraud in bank transactions, catching defects in manufacturing products, or even detecting network intrusions. In the context of credit card fraud detection, fraudulent transactions are rare compared to authentic transactions, so identifying these anomalies can be challenging. The approach I used, which involves detecting anomalies in a set of only negative examples, can be effective in identifying fraudulent transactions.  
Data address:
https://www.kaggle.com/datasets/whenamancodes/fraud-detection?select=creditcard.csv

# 1-Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, f1_score

# 2-Load the dataset

In [2]:
data_creditcard=pd.read_csv ('creditcard.csv')